# 注意力机制和Transformer
:label:`chap_attention-and-transformers`


深度学习热潮的最初几年主要由多层感知机、卷积网络和循环网络架构产生的结果推动。值得注意的是，尽管距离最初的模型已经过去了近30年，但在2010年代支撑许多深度学习突破的模型架构相对其前辈变化甚微。虽然许多新的方法创新进入了大多数实践者的工具箱——例如ReLU激活函数、残差层、批量归一化、dropout以及自适应学习率调度等——但核心底层架构显然是经典思想的放大实现。尽管有数千篇论文提出了替代方案，类似经典卷积神经网络（:numref:`chap_cnn`）的模型在计算机视觉中保持了*最先进*状态，而Sepp Hochreiter设计的原始LSTM循环神经网络（:numref:`sec_lstm`）则在自然语言处理中的大多数应用中占据主导地位。可以说，在那之前，深度学习的迅速出现主要归因于可用计算资源的变化（得益于GPU并行计算的创新）和大量数据资源的可用性（得益于廉价存储和互联网服务）。虽然这些因素可能仍然是这项技术日益强大的主要驱动力，但我们终于见证了一个主流架构领域的重大变革。

目前，几乎所有自然语言处理任务的主要模型都是基于Transformer架构。对于任何新的自然语言处理任务，默认的初步方法是抓取一个大型预训练的基于Transformer的模型（例如BERT :cite:`Devlin.Chang.Lee.ea.2018`，ELECTRA :cite:`clark2019electra`，RoBERTa :cite:`Liu.Ott.Goyal.ea.2019`或Longformer :cite:`beltagy2020longformer`），根据需要调整输出层，并在下游任务的可用数据上对模型进行微调。如果你在过去几年里一直关注OpenAI的大型语言模型相关的新闻报道，那么你一定跟踪了关于GPT-2和GPT-3基于Transformer的模型 :cite:`Radford.Wu.Child.ea.2019,brown2020language`为中心的讨论。与此同时，视觉Transformer已成为各种视觉任务的默认模型，包括图像识别、目标检测、语义分割和超分辨率 :cite:`Dosovitskiy.Beyer.Kolesnikov.ea.2021,liu2021swin`。Transformer也在语音识别 :cite:`gulati2020conformer`、强化学习 :cite:`chen2021decision`和图神经网络 :cite:`dwivedi2020generalization`中作为竞争方法出现。

Transformer模型的核心思想是*注意力机制*，这一创新最初设想为应用于序列到序列应用（如机器翻译 :cite:`Bahdanau.Cho.Bengio.2014`）的编码器-解码器RNN的增强。你可能还记得，在最早的机器翻译序列到序列模型 :cite:`Sutskever.Vinyals.Le.2014`中，整个输入被编码器压缩成一个固定长度的向量传递给解码器。注意力背后的直觉是，与其压缩输入，不如让解码器在每一步都重新访问输入序列。此外，解码器不应总是看到相同的输入表示，而是应该在特定解码步骤选择性地聚焦于输入序列的特定部分。Bahdanau的注意力机制提供了一种简单的方法，使解码器能够在每个解码步骤动态地*注意*输入的不同部分。高层次的想法是，编码器可以生成与原始输入序列长度相等的表示。然后，在解码时，解码器可以通过某种控制机制接收一个上下文向量作为输入，该向量由每个时间步输入表示的加权和组成。直观地说，权重决定了每个步骤的上下文对每个输入标记“关注”的程度，关键是使分配权重的过程可微分，以便与所有其他神经网络参数一起学习。

最初，这个想法作为对已经主导机器翻译应用的循环神经网络的一个显著成功的增强。这些模型的表现优于原来的编码器-解码器序列到序列架构。此外，研究人员注意到，有时从检查注意力权重的模式中会得出一些不错的定性见解。在翻译任务中，注意力模型通常会在生成目标语言相应词汇时对跨语言同义词赋予高注意力权重。例如，在将句子"my feet hurt"翻译成"j'ai mal au pieds"时，神经网络可能会在生成相应的法语单词"pieds"时对"feet"的表示赋予高注意力权重。这些见解促使人们声称注意力模型赋予了“可解释性”，尽管注意力权重的确切含义——即，如果有的话，它们应该如何被*解释*仍然是一片模糊的研究领域。

然而，注意力机制很快出现了超出其作为编码器-解码器循环神经网络增强及假设的选择显著输入功能之外的重大关切。:citet:`Vaswani.Shazeer.Parmar.ea.2017`提出了一种用于机器翻译的Transformer架构，完全摒弃了循环连接，而是依赖巧妙安排的注意力机制来捕捉所有输入和输出令牌之间的关系。该架构表现非常出色，到了2018年，Transformer开始出现在大多数最先进的自然语言处理系统中。此外，同一时期，自然语言处理中的主流做法成为在巨大的通用背景语料库上预训练大规模模型以优化某个自监督预训练目标，然后使用可用的下游数据对这些模型进行微调。当应用于这种预训练范式时，Transformer与传统架构之间的差距尤其大，因此Transformer的崛起与这样的大规模预训练模型（现在有时称为*基础模型* :cite:`bommasani2021opportunities`）的崛起同时发生。

在本章中，我们介绍注意力模型，从最基本直觉和最简单的实现开始。然后逐步深入到Transformer架构、视觉Transformer以及现代基于Transformer的预训练模型的景观。

:begin_tab:toc
 - [queries-keys-values](queries-keys-values.ipynb)
 - [attention-pooling](attention-pooling.ipynb)
 - [attention-scoring-functions](attention-scoring-functions.ipynb)
 - [bahdanau-attention](bahdanau-attention.ipynb)
 - [multihead-attention](multihead-attention.ipynb)
 - [self-attention-and-positional-encoding](self-attention-and-positional-encoding.ipynb)
 - [transformer](transformer.ipynb)
 - [vision-transformer](vision-transformer.ipynb)
 - [large-pretraining-transformers](large-pretraining-transformers.ipynb)
:end_tab: